In [13]:
# ==== IMPORT BIBLIOTECI ====
import ifcopenshell
import requests
import io
import traceback

def get_ifc_building_storeys(url):
    """
    Citește un fișier IFC de pe GitHub și extrage informații despre Building Storeys
    Optimizat pentru Google Colab
    """
    try:
        # Descărcare fișier IFC
        print("🔄 Descărcare fișier IFC...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        print(f"✅ Fișier descărcat cu succes! Dimensiune: {len(response.content)} bytes")

        # Citire directă în memorie
        print("🔄 Procesare fișier IFC...")
        # Save content to a temporary file since some IFC files may require file-based reading
        with open("temp.ifc", "wb") as f:
            f.write(response.content)

        model = ifcopenshell.open("temp.ifc")

        # Informații generale despre model
        print(f"\n📊 INFORMAȚII GENERALE:")
        print(f"Schema IFC: {model.schema}")
        print(f"Total entități: {len(list(model))}")  # Fixed: Use len(list(model)) to count entities

        # Extrage Building Storeys
        building_storeys = model.by_type("IfcBuildingStorey")

        print(f"\n🏢 BUILDING STOREYS GĂSITE: {len(building_storeys)}")
        print("=" * 60)

        if not building_storeys:
            print("⚠️ Nu au fost găsite Building Storeys în acest fișier IFC.")

            # Verifică alte tipuri de entități
            buildings = model.by_type("IfcBuilding")
            sites = model.by_type("IfcSite")
            spaces = model.by_type("IfcSpace")

            print(f"\n🔍 Alte entități disponibile:")
            print(f"   Buildings: {len(buildings)}")
            print(f"   Sites: {len(sites)}")
            print(f"   Spaces: {len(spaces)}")

            return

        # Afișează informații despre fiecare Building Storey
        for i, storey in enumerate(building_storeys, 1):
            print(f"\n🏗️ Building Storey {i}")
            print("-" * 40)
            print(f"GUID: {storey.GlobalId}")
            print(f"Nume: {storey.Name if storey.Name else '❌ N/A'}")
            print(f"Descriere: {storey.Description if storey.Description else '❌ N/A'}")
            print(f"LongName: {storey.LongName if storey.LongName else '❌ N/A'}")
            print(f"Elevation: {storey.Elevation if storey.Elevation is not None else '❌ N/A'}")

            # Extrage proprietățile dacă există
            properties_found = False
            if hasattr(storey, 'IsDefinedBy') and storey.IsDefinedBy:
                for rel in storey.IsDefinedBy:
                    if rel.is_a("IfcRelDefinesByProperties"):
                        prop_set = rel.RelatingPropertyDefinition
                        if prop_set.is_a("IfcPropertySet"):
                            if not properties_found:
                                print("📋 Proprietăți:")
                                properties_found = True
                            print(f"  📦 PropertySet: {prop_set.Name}")
                            if hasattr(prop_set, 'HasProperties'):
                                for prop in prop_set.HasProperties:
                                    if hasattr(prop, 'Name') and hasattr(prop, 'NominalValue'):
                                        value = prop.NominalValue.wrappedValue if prop.NominalValue else 'N/A'
                                        print(f"    • {prop.Name}: {value}")

            # Verifică dacă etajul conține spații
            spaces = []
            if hasattr(storey, 'IsDecomposedBy'):
                for rel in storey.IsDecomposedBy:
                    if rel.is_a("IfcRelAggregates"):
                        for obj in rel.RelatedObjects:
                            if obj.is_a("IfcSpace"):
                                spaces.append(obj.Name if obj.Name else "Spațiu fără nume")

            if spaces:
                print(f"🏠 Spații conținute ({len(spaces)}): {', '.join(spaces)}")
            else:
                print("🏠 Spații conținute: ❌ Niciunul")

        # Statistici generale
        print(f"\n📈 STATISTICI GENERALE")
        print("=" * 60)
        print(f"🏢 Total Building Storeys: {len(building_storeys)}")

        # Calculează numărul total de spații
        total_spaces = 0
        for storey in building_storeys:
            if hasattr(storey, 'IsDecomposedBy'):
                for rel in storey.IsDecomposedBy:
                    if rel.is_a("IfcRelAggregates"):
                        for obj in rel.RelatedObjects:
                            if obj.is_a("IfcSpace"):
                                total_spaces += 1

        print(f"🏠 Total spații în toate etajele: {total_spaces}")

        # Afișează elevațiile sortate
        elevations = []
        for storey in building_storeys:
            if storey.Elevation is not None:
                elevations.append((storey.Name if storey.Name else "Fără nume", storey.Elevation))

        if elevations:
            elevations.sort(key=lambda x: x[1])  # Sortează după elevație
            print(f"\n📏 Etaje sortate după elevație:")
            for name, elevation in elevations:
                print(f"  🔸 {name}: {elevation}")

        print("\n✅ Procesare completă!")

    except requests.RequestException as e:
        print(f"❌ Eroare la descărcarea fișierului: {e}")
        print("💡 Verifică dacă URL-ul este corect și accesibil")
    except Exception as e:
        print(f"❌ Eroare la procesarea fișierului IFC: {e}")
        print("🔍 Detalii complete ale erorii:")
        print(traceback.format_exc())

# ==== UTILIZARE ====
# Înlocuiește cu URL-ul real al fișierului IFC
url = "https://raw.githubusercontent.com/ionuting/AwsomeViewer/refs/heads/main/model.ifc"


# Fișier IFC mai complex
# url = "https://raw.githubusercontent.com/buildingSMART/Sample-Test-Files/master/IFC%202x3/Schependomlaan/IFC%20Schependomlaan.ifc"

print("🚀 Începere procesare fișier IFC...")
print(f"📂 URL: {url}")
print("-" * 80)

get_ifc_building_storeys(url)

🚀 Începere procesare fișier IFC...
📂 URL: https://raw.githubusercontent.com/ionuting/AwsomeViewer/refs/heads/main/model.ifc
--------------------------------------------------------------------------------
🔄 Descărcare fișier IFC...
✅ Fișier descărcat cu succes! Dimensiune: 429112 bytes
🔄 Procesare fișier IFC...

📊 INFORMAȚII GENERALE:
Schema IFC: IFC4
Total entități: 6395

🏢 BUILDING STOREYS GĂSITE: 5

🏗️ Building Storey 1
----------------------------------------
GUID: 2YmaQ8oXj9A8Fo5t$6NGj7
Nume: UG
Descriere: ❌ N/A
LongName: ❌ N/A
Elevation: -3.4
📋 Proprietăți:
  📦 PropertySet: Pset_SpatialData
    • BuildingStory: UG
    • ElevationOfStory: -3.4
    • BuildingStory: EG
    • ElevationOfStory: 0.0
    • BuildingStory: O1
    • ElevationOfStory: 3.4
    • BuildingStory: O2
    • ElevationOfStory: 6.9
    • BuildingStory: Storey 6900
    • ElevationOfStory: 6.9
🏠 Spații conținute (4): SGR, GrossVolume, GrossVolume, GrossArea

🏗️ Building Storey 2
---------------------------------------

In [8]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# ==== INSTALARE DEPENDENȚE ====
# Rulează această celulă prima dată
!pip install ifcopenshell requests plotly ipywidgets


In [25]:
# ==== IMPORT BIBLIOTECI ====
import ifcopenshell
import requests
import json
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
import ipywidgets as widgets

def get_ifc_building_storeys_and_visualize(url):
    """
    Citește un fișier IFC de pe GitHub, extrage informații despre Building Storeys și vizualizează mesh-urile din Custom_Mesh JSON cu Plotly.
    Transformă coordonatele și rotațiile pentru a alinia axa Z (verticală) a IFC cu axa Y (verticală) a Plotly, asigurând geometria upright.
    Adaugă interacțiune: click pe obiect sau checkbox pentru selecție (schimbă culoare, afișează proprietăți) și toggle vizibilitate.
    Optimizat pentru Google Colab.
    """
    try:
        # Descărcare fișier IFC
        print("🔄 Descărcare fișier IFC...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        print(f"✅ Fișier descărcat cu succes! Dimensiune: {len(response.content)} bytes")

        # Citire directă în memorie
        print("🔄 Procesare fișier IFC...")
        with open("temp.ifc", "wb") as f:
            f.write(response.content)

        model = ifcopenshell.open("temp.ifc")

        # Informații generale despre model
        print(f"\n📊 INFORMAȚII GENERALE:")
        print(f"Schema IFC: {model.schema}")
        print(f"Total entități: {len(list(model))}")

        # Extrage Building Storeys
        building_storeys = model.by_type("IfcBuildingStorey")

        print(f"\n🏢 BUILDING STOREYS GĂSITE: {len(building_storeys)}")
        print("=" * 60)

        if not building_storeys:
            print("⚠️ Nu au fost găsite Building Storeys în acest fișier IFC.")

            # Verifică alte tipuri de entități
            buildings = model.by_type("IfcBuilding")
            sites = model.by_type("IfcSite")
            spaces = model.by_type("IfcSpace")

            print(f"\n🔍 Alte entități disponibile:")
            print(f"   Buildings: {len(buildings)}")
            print(f"   Sites: {len(sites)}")
            print(f"   Spaces: {len(spaces)}")

        # Afișează informații despre fiecare Building Storey
        for i, storey in enumerate(building_storeys, 1):
            print(f"\n🏗️ Building Storey {i}")
            print("-" * 40)
            print(f"GUID: {storey.GlobalId}")
            print(f"Nume: {storey.Name if storey.Name else '❌ N/A'}")
            print(f"Descriere: {storey.Description if storey.Description else '❌ N/A'}")
            print(f"LongName: {storey.LongName if storey.LongName else '❌ N/A'}")
            print(f"Elevation: {storey.Elevation if storey.Elevation is not None else '❌ N/A'}")

            # Extrage proprietățile dacă există
            properties_found = False
            if hasattr(storey, 'IsDefinedBy') and storey.IsDefinedBy:
                for rel in storey.IsDefinedBy:
                    if rel.is_a("IfcRelDefinesByProperties"):
                        prop_set = rel.RelatingPropertyDefinition
                        if prop_set.is_a("IfcPropertySet"):
                            if not properties_found:
                                print("📋 Proprietăți:")
                                properties_found = True
                            print(f"  📦 PropertySet: {prop_set.Name}")
                            if hasattr(prop_set, 'HasProperties'):
                                for prop in prop_set.HasProperties:
                                    if hasattr(prop, 'Name') and hasattr(prop, 'NominalValue'):
                                        value = prop.NominalValue.wrappedValue if prop.NominalValue else 'N/A'
                                        print(f"    • {prop.Name}: {value}")

            # Verifică dacă etajul conține elemente
            elements = []
            if hasattr(storey, 'ContainsElements'):
                for rel in storey.ContainsElements:
                    if rel.is_a("IfcRelContainedInSpatialStructure"):
                        for obj in rel.RelatedElements:
                            if obj.is_a("IfcSpace"):
                                elements.append(obj.Name if obj.Name else "Spațiu fără nume")
                            else:
                                elements.append(f"{obj.is_a()}: {obj.Name if obj.Name else 'Fără nume'}")

            if elements:
                print(f"🏠 Elemente conținute ({len(elements)}): {', '.join(elements)}")
            else:
                print("🏠 Elemente conținute: ❌ Niciunul")

        # Statistici generale
        print(f"\n📈 STATISTICI GENERALE")
        print("=" * 60)
        print(f"🏢 Total Building Storeys: {len(building_storeys)}")

        # Calculează numărul total de elemente
        total_elements = 0
        for storey in building_storeys:
            if hasattr(storey, 'ContainsElements'):
                for rel in storey.ContainsElements:
                    if rel.is_a("IfcRelContainedInSpatialStructure"):
                        total_elements += len(rel.RelatedElements)

        print(f"🏠 Total elemente în toate etajele: {total_elements}")

        # Afișează elevațiile sortate
        elevations = []
        for storey in building_storeys:
            if storey.Elevation is not None:
                elevations.append((storey.Name if storey.Name else "Fără nume", storey.Elevation))

        if elevations:
            elevations.sort(key=lambda x: x[1])
            print(f"\n📏 Etaje sortate după elevație:")
            for name, elevation in elevations:
                print(f"  🔸 {name}: {elevation}")

        # Extrage și vizualizează mesh-urile din Custom_Mesh
        print("\n🔍 Extragere mesh-uri din Custom_Mesh...")
        mesh_json = None
        for entity in model.by_type("IfcWall") + model.by_type("IfcColumn"):
            for rel in entity.IsDefinedBy:
                if rel.RelatingPropertyDefinition.Name == "Pset_CustomGeometry":
                    for prop in rel.RelatingPropertyDefinition.HasProperties:
                        if prop.Name == "Custom_Mesh":
                            mesh_json = prop.NominalValue.wrappedValue
                            break
                    if mesh_json:
                        break
            if mesh_json:
                break

        if not mesh_json:
            print("⚠️ Nu s-a găsit Custom_Mesh în fișierul IFC.")
            return

        print("✅ Mesh JSON găsit! Procesare pentru vizualizare...")
        mesh_data = json.loads(mesh_json)

        # Creează figuri Plotly
        fig = go.Figure()
        mesh_dict = {}  # Pentru a mapa mesh-urile după nume
        original_colors = {}  # Pentru a păstra culorile originale
        visibility = {}  # Pentru a urmări vizibilitatea
        properties = {}  # Pentru a stoca proprietățile

        for element in mesh_data["elements"]:
            mesh_id = element["mesh_id"]
            mesh_info = next((m for m in mesh_data["meshes"] if m["mesh_id"] == mesh_id), None)
            if not mesh_info:
                continue

            # Extrage vertices și indices
            vertices = np.array(mesh_info["coordinates"], dtype=np.float32).reshape(-1, 3)

            # Aplică rotația în coordonatele IFC (Z vertical)
            if element["rotation"]:
                from scipy.spatial.transform import Rotation
                # Quaternion în coordonatele IFC: (qw, qx, qy, qz)
                quat = [element["rotation"]["qw"], element["rotation"]["qx"], element["rotation"]["qy"], element["rotation"]["qz"]]
                rot = Rotation.from_quat(quat)
                vertices = rot.apply(vertices)

            # Transformă coordonatele: IFC (x, y, z) → Plotly (x, z, y)
            vertices = vertices[:, [0, 2, 1]]  # x_ifc → x, z_ifc → y, y_ifc → z

            # Aplică translația în coordonatele Plotly
            position = [0, 0, 0]
            if element["vector"]:
                # Transformă vectorul: IFC (x, y, z) → Plotly (x, z, y)
                position = [element["vector"]["x"], element["vector"]["z"], element["vector"]["y"]]
            vertices += position  # Aplică translația

            indices = np.array(mesh_info["indices"], dtype=np.uint32)

            # Creează mesh-ul Plotly
            r, g, b = element["color"]["r"], element["color"]["g"], element["color"]["b"]
            hex_color = f"#{r:02x}{g:02x}{b:02x}"
            name = element["info"]["Name"]

            mesh = go.Mesh3d(
                x=vertices[:, 0],
                y=vertices[:, 1],
                z=vertices[:, 2],
                i=indices[0::3],
                j=indices[1::3],
                k=indices[2::3],
                color=hex_color,
                opacity=element["color"]["a"] / 255,
                name=name,
                visible=True
            )

            fig.add_trace(mesh)
            mesh_dict[name] = mesh
            original_colors[name] = hex_color
            visibility[name] = True
            properties[name] = element["info"]

        # Configurează layout-ul Plotly cu camera ajustată
        fig.update_layout(
            scene=dict(
                xaxis=dict(title="X"),
                yaxis=dict(title="Y (Vertical)"),
                zaxis=dict(title="Z"),
                aspectmode="data",
                camera=dict(
                    up=dict(x=0, y=1, z=0),  # Y este vertical
                    eye=dict(x=1.5, y=1.5, z=1.5)  # Perspectivă izometrică
                )
            ),
            showlegend=False,
            width=800,
            height=600,
            margin=dict(l=0, r=200, t=0, b=0)
        )

        # Panou de proprietăți
        properties_panel = widgets.HTML(value="<b>Proprietăți:</b><br>Selectați un obiect")

        # Panou de vizibilitate și selecție
        visibility_checkboxes = []
        selected_mesh = [None]  # Pentru a urmări mesh-ul selectat

        def update_visibility_and_selection(change):
            name = change["owner"].description
            visibility[name] = change["new"]
            mesh_dict[name].visible = change["new"]

            # Dacă checkbox-ul este bifat, selectează mesh-ul
            if change["new"]:
                selected_mesh[0] = name
                for mesh_name in mesh_dict:
                    mesh_dict[mesh_name].color = "#ffff00" if mesh_name == name else original_colors[mesh_name]
                props_html = "<b>Proprietăți:</b><br>"
                for key, value in properties[name].items():
                    props_html += f"{key}: {value}<br>"
                properties_panel.value = props_html
            else:
                if selected_mesh[0] == name:
                    selected_mesh[0] = None
                    properties_panel.value = "<b>Proprietăți:</b><br>Selectați un obiect"
                    for mesh_name in mesh_dict:
                        mesh_dict[mesh_name].color = original_colors[mesh_name]

            # Actualizează figura
            fig.update_traces(selector=dict(name=name), visible=visibility[name])

        for name in mesh_dict:
            checkbox = widgets.Checkbox(
                value=True,
                description=name,
                indent=False
            )
            checkbox.observe(update_visibility_and_selection, names="value")
            visibility_checkboxes.append(checkbox)

        visibility_panel = widgets.VBox(
            [widgets.Label("Vizibilitate și Selecție Obiecte:")] + visibility_checkboxes,
            layout={'border': '1px solid black', 'padding': '10px', 'width': '200px', 'max_height': '200px', 'overflow': 'auto'}
        )

        # Funcție pentru gestionarea selecției prin click
        def on_click(trace, points, selector):
            if points.trace_name:
                name = points.trace_name
                selected_mesh[0] = name
                for mesh_name in mesh_dict:
                    mesh_dict[mesh_name].color = "#ffff00" if mesh_name == name else original_colors[mesh_name]
                    fig.update_traces(selector=dict(name=mesh_name), color=mesh_dict[mesh_name].color)
                props_html = "<b>Proprietăți:</b><br>"
                for key, value in properties[name].items():
                    props_html += f"{key}: {value}<br>"
                properties_panel.value = props_html
                # Sincronizează checkbox-urile
                for checkbox in visibility_checkboxes:
                    if checkbox.description == name:
                        checkbox.value = True
                    elif selected_mesh[0] != checkbox.description:
                        checkbox.value = visibility[checkbox.description]

        for trace in fig.data:
            trace.on_click(on_click)

        # Afișează interfața
        print("\n🎨 Afișare vizualizare 3D și panouri interactive...")
        # Afișează figura Plotly separat
        fig.show(renderer="colab")
        # Afișează panourile de control
        display(widgets.VBox([properties_panel, visibility_panel], layout={'margin': '10px'}))

        print("\n✅ Procesare completă!")

    except requests.RequestException as e:
        print(f"❌ Eroare la descărcarea fișierului: {e}")
        print("💡 Verifică dacă URL-ul este corect și accesibil")
    except Exception as e:
        print(f"❌ Eroare la procesarea fișierului IFC sau vizualizare: {e}")
        print("🔍 Detalii complete ale erorii:")
        import traceback
        print(traceback.format_exc())

# ==== UTILIZARE ====
# URL-ul fișierului IFC
url = "https://raw.githubusercontent.com/ionuting/AwsomeViewer/refs/heads/main/output1.ifc"

print("🚀 Începere procesare fișier IFC...")
print(f"📂 URL: {url}")
print("-" * 80)

get_ifc_building_storeys_and_visualize(url)

🚀 Începere procesare fișier IFC...
📂 URL: https://raw.githubusercontent.com/ionuting/AwsomeViewer/refs/heads/main/output1.ifc
--------------------------------------------------------------------------------
🔄 Descărcare fișier IFC...
✅ Fișier descărcat cu succes! Dimensiune: 4885 bytes
🔄 Procesare fișier IFC...

📊 INFORMAȚII GENERALE:
Schema IFC: IFC4
Total entități: 30

🏢 BUILDING STOREYS GĂSITE: 1

🏗️ Building Storey 1
----------------------------------------
GUID: 9085cdfa-4787-4949-95ae-00185d8acdfe
Nume: Level 1
Descriere: ❌ N/A
LongName: ❌ N/A
Elevation: 0.0
🏠 Elemente conținute (2): IfcWall: Wall_1, IfcColumn: Column_1

📈 STATISTICI GENERALE
🏢 Total Building Storeys: 1
🏠 Total elemente în toate etajele: 2

📏 Etaje sortate după elevație:
  🔸 Level 1: 0.0

🔍 Extragere mesh-uri din Custom_Mesh...
✅ Mesh JSON găsit! Procesare pentru vizualizare...

🎨 Afișare vizualizare 3D și panouri interactive...



✅ Procesare completă!
